# Import Libraries

In [1]:
import mysql.connector as sql
import pandas as pd

# Connect to Sql Databse

In [2]:
db_connection = sql.connect(host="localhost",database="country_club",user='root',password='')
db_cursor = db_connection.cursor()


Q1: Some of the facilities charge a fee to members, but some do not.
Please list the names of the facilities that do.

In [3]:
db_cursor.execute('SELECT * FROM facilities WHERE membercost > 0')
table_rows = db_cursor.fetchall()
df = pd.DataFrame(table_rows,columns = ['facid','name','membercost','guestcost', 'initialoutlay','monthlymaintenance'])
df

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,4,Massage Room 1,9.9,80.0,4000,3000
3,5,Massage Room 2,9.9,80.0,4000,3000
4,6,Squash Court,3.5,17.5,5000,80


 Q2: How many facilities do not charge a fee to members?

In [4]:
db_cursor.execute('SELECT COUNT(*) FROM facilities WHERE membercost = 0')
table_rows = db_cursor.fetchall()
df = pd.DataFrame(table_rows,columns=['Count'])
df

,Count
0,4


Q3: How can you produce a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost?
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question

In [5]:
db_cursor.execute('SELECT facid, name, membercost, monthlymaintenance FROM facilities WHERE membercost > 0 AND(membercost < monthlymaintenance * 0.20)')
table_rows = db_cursor.fetchall()
df = pd.DataFrame(table_rows,columns=['facid','name','membercost','monthlymaintenance'])
df

,facid,name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,4,Massage Room 1,9.9,3000
3,5,Massage Room 2,9.9,3000
4,6,Squash Court,3.5,80


Q4: How can you retrieve the details of facilities with ID 1 and 5?
Write the query without using the OR operator.

In [6]:
db_cursor.execute('SELECT * FROM `Facilities` where facid in (1,5)')
table_rows = db_cursor.fetchall()
df = pd.DataFrame(table_rows,columns=['facid','name','membercost','guestcost', 'initialoutlay','monthlymaintenance'])
df

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25.0,8000,200
1,5,Massage Room 2,9.9,80.0,4000,3000


Q5: How can you produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100? Return the name and monthly maintenance of the facilities
in question.

In [7]:
db_cursor.execute('Select name,IF(monthlymaintenance < 100, "Cheap","Expensive") from Facilities;')
table_rows = db_cursor.fetchall()
df = pd.DataFrame(table_rows,columns=['name','monthlymaintenance'])
df

,name,monthlymaintenance
0,Tennis Court 1,Expensive
1,Tennis Court 2,Expensive
2,Badminton Court,Cheap
3,Table Tennis,Cheap
4,Massage Room 1,Expensive
5,Massage Room 2,Expensive
6,Squash Court,Cheap
7,Snooker Table,Cheap
8,Pool Table,Cheap


Q6: You'd like to get the first and last name of the last member(s)
who signed up. Do not use the LIMIT clause for your solution.

In [8]:
db_cursor.execute('select firstname,Surname from Members where joindate= (select MAX(joindate) from Members);')
table_rows = db_cursor.fetchall()
df = pd.DataFrame(table_rows,columns=['firstname','Surname'])
df

,firstname,Surname
0,Darren,Smith


Q7: How can you produce a list of all members who have used a tennis court?
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.

In [9]:
db_cursor.execute('SELECT DISTINCT firstname || " " || surname AS member, name AS facility FROM members INNER JOIN bookings ON members.memid = bookings.memid INNER JOIN facilities ON bookings.facid = facilities.facid WHERE name LIKE "%Tennis Court%" ORDER BY member')
table_rows = db_cursor.fetchall()
df = pd.DataFrame(table_rows,columns=['member','name'])
df

,member,name
0,0,Tennis Court 1
1,0,Tennis Court 2


Q8: How can you produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30? Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.

In [10]:
db_cursor.execute('SELECT mem.firstname|| " " ||mem.surname AS member,fac.name AS facility,\
CASE WHEN firstname = "GUEST" THEN (fac.guestcost * bk.slots) ELSE (fac.membercost * bk.slots )END AS cost \
FROM members mem \
INNER JOIN bookings bk \
ON mem.memid = bk.memid \
INNER JOIN facilities fac \
ON bk.facid = fac.facid \
WHERE starttime >= "2012-09-14" AND starttime < "2012-09-15" \
AND CASE WHEN mem.firstname = "GUEST" THEN (fac.guestcost * bk.slots) ELSE (fac.membercost * bk.slots) END > 30 \
ORDER BY cost DESC')
table_rows = db_cursor.fetchall()
df = pd.DataFrame(table_rows)
df

""


Q9: This time, produce the same result as in Q8, but using a subquery.

In [11]:
db_cursor.execute('SELECT firstname || " " || surname AS member ,name AS facility,cost FROM  (SELECT firstname, surname, name,\
CASE WHEN firstname = "GUEST" THEN (guestcost * slots ) ELSE (membercost * slots) END AS cost, \
starttime \
FROM members \
INNER JOIN bookings \
ON members.memid = bookings.memid \
INNER JOIN facilities \
ON bookings.facid = facilities.facid) AS inner_table \
WHERE starttime >= "2012-09-14" AND starttime < "2012-09-15" \
AND cost > 30 \
ORDER BY cost DESC;')
table_rows = db_cursor.fetchall()
df = pd.DataFrame(table_rows)
df

""


Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! 


In [12]:
db_cursor.execute('SELECT fac.name,fac.facid,SUM(CASE WHEN mem.memid = 0 THEN fac.guestcost * book.slots ELSE fac.membercost * book.slots END) AS revenue \
FROM `Facilities` fac \
JOIN `Bookings` book ON fac.facid = book.facid \
JOIN `Members` mem ON book.memid = mem.memid \
GROUP BY fac.name, fac.facid \
HAVING revenue > 1000 \
ORDER BY revenue') 
table_rows = db_cursor.fetchall() 
df = pd.DataFrame(table_rows,columns=['name','facid','revenue'])
df

,name,facid,revenue
0,Massage Room 2,5,3338.6
1,Squash Court,6,4158.0
2,Tennis Court 1,0,5025.0
3,Tennis Court 2,1,5310.0
4,Massage Room 1,4,17269.4
